In [2]:
from re import sub
from os import listdir
from collections import Counter
from itertools import chain
from numpy import array
from jieba import cut
from sklearn.naive_bayes import MultinomialNB
import numpy as np

def getWordsFromFile(txtFile):
    # 获取每一封邮件中的所有词语
    words = []
    # 所有存储邮件文本内容的记事本文件都使用UTF8编码
    with open(txtFile, encoding='utf8') as fp:
        for line in fp:
            # 遍历每一行，删除两端的空白字符
            line = line.strip()
            # 过滤干扰字符或无效字符
            line = sub(r'[.【】0-9、—。，！~\*]', '', line)
            # 分词
            line = cut(line)
            # 过滤长度为1的词
            line = filter(lambda word: len(word)>1, line)
            # 把本行文本预处理得到的词语添加到words列表中
            words.extend(line)
    # 返回包含当前邮件文本中所有有效词语的列表
    return words

# 存放所有文件中的单词
# 每个元素是一个子列表，其中存放一个文件中的所有单词
allWords = []
def getTopNWords(topN):
    # 按文件编号顺序处理当前文件夹中所有记事本文件
    # 训练集中共含有多封邮件内容，spam_data.txt是垃圾邮件内容,ham_data.txt是正常邮件
    txtFiles = ['spam_data.txt','ham_data.txt']
    # 获取训练集中所有邮件中的全部单词
    for txtFile in txtFiles:
        allWords.append(getWordsFromFile(txtFile))
    # 获取并返回出现次数最多的前topN个单词
    freq = Counter(chain(*allWords))
    return [w[0] for w in freq.most_common(topN)]

N = np.random.randint(600,1001)
# 全部训练集中出现次数最多的前600个单词
topWords = getTopNWords(N)

# 获取特征向量，前N个单词的每个单词在每个邮件中出现的频率
vectors = []
for words in allWords:
    temp = list(map(lambda x: words.count(x), topWords))
    vectors.append(temp)

# 训练集中每个邮件的标签，1表示垃圾邮件，0表示正常邮件
labels = [1,0]

# 创建模型，使用已知训练集进行训练
model = MultinomialNB()
model.fit(vectors, labels)

def predict(txtFile):
    # 获取指定邮件文件内容，返回分类结果
    words = getWordsFromFile(txtFile)
    currentVector = array(tuple(map(lambda x: words.count(x),
                                    topWords)))
    result = model.predict(currentVector.reshape(1, -1))[0]
    print(model.predict_proba(currentVector.reshape(1, -1)))
    return '垃圾邮件' if result==1 else '正常邮件'

# 随机选取部分邮件作为测试邮件内容
for mail in ('%d.txt'%i for i in range(np.random.randint(1,75), np.random.randint(75,150))):
    print(mail, predict(mail), sep=':')


训练集精度: 1.0
[[9.99980022e-01 1.99778236e-05]]
28.txt:正常邮件
[[3.41001642e-19 1.00000000e+00]]
29.txt:垃圾邮件
[[7.50458975e-07 9.99999250e-01]]
30.txt:垃圾邮件
[[9.99999979e-01 2.05054531e-08]]
31.txt:正常邮件
[[9.99984742e-01 1.52582713e-05]]
32.txt:正常邮件
[[1.07129285e-07 9.99999893e-01]]
33.txt:垃圾邮件
[[7.29689915e-13 1.00000000e+00]]
34.txt:垃圾邮件
[[1.10899173e-15 1.00000000e+00]]
35.txt:垃圾邮件
[[1.32086709e-04 9.99867913e-01]]
36.txt:垃圾邮件
[[3.14066745e-08 9.99999969e-01]]
37.txt:垃圾邮件
[[1.02648740e-06 9.99998974e-01]]
38.txt:垃圾邮件
[[1.02152828e-06 9.99998978e-01]]
39.txt:垃圾邮件
[[1.00000000e+00 1.39828559e-13]]
40.txt:正常邮件
[[1.0000000e+00 1.5715467e-15]]
41.txt:正常邮件
[[1.19933827e-08 9.99999988e-01]]
42.txt:垃圾邮件
[[1.02648740e-06 9.99998974e-01]]
43.txt:垃圾邮件
[[1.51398285e-06 9.99998486e-01]]
44.txt:垃圾邮件
[[5.97653015e-08 9.99999940e-01]]
45.txt:垃圾邮件
[[1.83829175e-22 1.00000000e+00]]
46.txt:垃圾邮件
[[0.95521816 0.04478184]]
47.txt:正常邮件
[[0.99567427 0.00432573]]
48.txt:正常邮件
[[2.98547379e-12 1.00000000e+00]]
49.txt: